# UK Road Safety: Traffic Accidents and Vehicles - FLASK Web Server

## The following code implements the standalone Flask Server Application that can either be run locally or on a remote server
It requires setting up VCAP services to access the Cloudant database from the server<br>
Currently this is run on my local machine - not the Spark cluster<br>
Code given for illustration<br>
The set-up for this app was inspired from:
https://github.com/IBM-Cloud/get-started-python

In [ ]:
from cloudant import Cloudant
from cloudant.error import CloudantException
from cloudant.result import Result, ResultByKey
from cloudant.document import Document
from simplejson.decoder import JSONDecoder as json_decoder
import base64, io, os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from flask import Flask, render_template, request, jsonify
import atexit
import json
from flask import make_response, url_for


my_path = "/Users/ensemble/Documents/Coursera/Capstone/Web_App/static/"
#app = Flask(__name__, static_url_path='/Users/ensemble/Documents/Coursera/Capstone/Web_App/static')
app = Flask(__name__, static_url_path='')
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0

client = None
db = None


#Connect to the Cloudant Database
if 'VCAP_SERVICES' in os.environ:
    vcap = json.loads(os.getenv('VCAP_SERVICES'))
    print('Found VCAP_SERVICES')
    if 'cloudantNoSQLDB' in vcap:
        creds = vcap['cloudantNoSQLDB'][0]['credentials']
        user = creds['username']
        password = creds['password']
        url = 'https://' + creds['host']
        client = Cloudant(user, password, url=url, connect=True)
        db = client["map_query"]
elif "CLOUDANT_URL" in os.environ:
    client = Cloudant(os.environ['CLOUDANT_USERNAME'], 
                      os.environ['CLOUDANT_PASSWORD'], 
                      url=os.environ['CLOUDANT_URL'], connect=True)
    db = client["map_query"]
    #db = client.create_database(db_name, throw_on_exists=False)
elif os.path.isfile('vcap-local.json'):
    with open('vcap-local.json') as f:
        vcap = json.load(f)
        print('Found local VCAP_SERVICES')
        creds = vcap['services']['cloudantNoSQLDB'][0]['credentials']
        user = creds['username']
        password = creds['password']
        url = 'https://' + creds['host']
        client = Cloudant(user, password, url=url, connect=True)
        db = client["map_query"]

# On IBM Cloud Cloud Foundry, get the port number from the environment variable PORT
# When running this app on the local machine, default the port to 8000
port = int(os.getenv('PORT', 8000))

db_pf  = client["police_force"]
cat_pf  = client["cat_mappings"]

pf_list = list(map(lambda doc: doc['pf_list'], db_pf))[0]
cat_names = list(map(lambda doc: doc['name'], cat_pf))
cat_names = [cat.replace('_', ' ') for cat in cat_names]
cat_values = list(map(lambda doc: doc['values'], cat_pf))

#load a query from the database
def load_queryfrom_cdB(query_db):
    json_doc = Result(query_db.all_docs, include_docs=True)[0][0]['doc']
    json_doc.pop('_rev')
    query_id = json_doc['_id']
    return query_id, json_doc

#Setting up some global variables
query_id, json_doc = load_queryfrom_cdB(db)  
pf_selected = json_doc['Police_Force']
rmse = json_doc['rmse']
R2 = json_doc['R2']
cat_selected = [json_doc[cat] for cat in [cat.replace(' ', '_') for cat in cat_names]]


#Read map attachement in the database entry (PNG file)
def get_attachement(query_db):
    query_id, query_dict = load_queryfrom_cdB(query_db)
    doc = Document(query_db, query_id)#, decoder=json_decoder)
    return doc.get_attachment('map_img.png')

#create a new database 
def create_database(db_name):
    database = client.create_database(db_name, throw_on_exists=False)
    if database.exists():
        print("{} database successfully created".format(db_name))
        return database
    else:
        return None
    
#delete database
def delete_database(db_name):
    try :
        client.delete_database(db_name)
    except CloudantException:
        print("There was a problem deleting {}.\n".format(db_name))
    else:
        print("{} successfully deleted.\n".format(db_name))
    return
 
#create a new database to hold the query (the database only holds one query at a time 
#and is recreated for  each new query)
def create_temp_database(db_name):
    try :
        client.delete_database(db_name)
    except CloudantException:
        pass
    return create_database(db_name)

#save query to db
def save_map_query_to_cdB(query_dict):
    query_db = create_temp_database("map_query")
    doc = query_db.create_document(query_dict)
    return query_db, doc


#load query from db
def load_details_from_query(db):
    query_id, json_doc = load_queryfrom_cdB(db)  
    pf_selected = json_doc['Police_Force']
    rmse = json_doc['rmse']
    R2 = json_doc['R2']
    cat_selected = [json_doc[cat] for cat in [cat.replace(' ', '_') for cat in cat_names]]
    return

#load map image, decode saves it to the local 'static' folder and returns a matplotlib fig object
def get_and_save_map(db):
    try:
        img = get_attachement(db)
        img = base64.b64decode(img)
        img = io.BytesIO(img)
        img = mpimg.imread(img, format='PNG')
        fig, ax = plt.subplots()
        ax.imshow(img, interpolation='nearest')
        plt.tight_layout()
        ax.grid('off')
        plt.axis('equal')
        ax.axis('off')
        fig.savefig(my_path+"map.png", format='png', dpi=300)
        plt.close(fig)
        return fig
    except:
        return None


#overides Flask url_for function to ignore the cache 
#otherwise map will never update since we always keep the same file name for the map in the static folder
@app.context_processor
def override_url_for():
    return dict(url_for=dated_url_for)

def dated_url_for(endpoint, **values):
    if endpoint == 'static':
        filename = values.get('filename', None)
        if filename:
            file_path = os.path.join(app.root_path,
                                     endpoint, filename)
            values['q'] = int(os.stat(file_path).st_mtime)
    return url_for(endpoint, **values)



#define route for start page
@app.route('/')
def root():
    get_and_save_map(db) 
    load_details_from_query(db)
    #path_to_img = os.path.join(my_path, 'map.png')
    path_to_img='map.png'
    response = make_response(render_template('index.html', title='Main', \
            pf_list=pf_list, pf_sel=pf_selected, cat_names=cat_names, cat_values=cat_values, \
            cat_sel=cat_selected, img=path_to_img, rmse="{:.2f}".format(rmse), R2="{:.2f}".format(R2)))
    return response


#define route for run_model page
@app.route('/run_model', methods=['POST','GET'])
def run_model():
    pf_selected = request.form["police"]
    cat_selected = [request.form["cat"+str(i)] for i in range(5)]
    path_to_img='map.png'
    web_query = {}
    for i, cat in enumerate([cat.replace(' ', '_') for cat in cat_names]):
        web_query[cat] = cat_selected[i]
    web_query['LSOA_of_Accident_Location'] = ''
    web_query['Centroid_Latitude'] = 0 
    web_query['Centroid_Longitude'] = 0
    web_query['rmse'] = 0 
    web_query['R2'] = 0
    web_query['Police_Force'] = pf_selected
    db, doc = save_map_query_to_cdB(web_query)
    while get_and_save_map(db)==None:
        print("Processing....")
    load_details_from_query(db)  
    response = make_response(render_template('index.html', title='Main', \
            pf_list=pf_list, pf_sel=pf_selected,cat_names=cat_names, cat_values=cat_values, \
            cat_sel=cat_selected, img=path_to_img, rmse="{:.2f}".format(rmse), R2="{:.2f}".format(R2)))   
    return response

#Close the connection
@atexit.register
def shutdown():
    if client:
        client.disconnect()

#Main
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=port, debug=True)


## The following is the HTML code for the WEB form

In [ ]:
<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <title>Accident Severity Prediction App</title>

    <!-- Bootstrap -->
    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css" rel="stylesheet">
    <link href="styles.css" rel="stylesheet">
    <!-- jQuery (necessary for Bootstrap's JavaScript plugins) -->
    <script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
    <!-- Include all compiled plugins (below), or include individual files as needed -->
    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
    <script src="js/lib/jquery.i18n/jquery.i18n.js"></script>
    <script src="js/lib/jquery.i18n/jquery.i18n.messagestore.js"></script>
    <script src="js/lib/jquery.i18n/jquery.i18n.fallbacks.js"></script>
    <script src="js/lib/jquery.i18n/jquery.i18n.language.js"></script>
    <script src="js/lib/jquery.i18n/jquery.i18n.parser.js"></script>
    <script src="js/lib/jquery.i18n/jquery.i18n.emitter.js"></script>
    <script src="js/lib/jquery.i18n/jquery.i18n.emitter.bidi.js"></script>
    <script src="antixss.js" type="text/javascript"></script>

    <script>
        $( document ).ready(function() {
            $.i18n().load( {
                en: {
                    "welcome": "Welcome.",
                    "name": "name",
                    "what_is_your_name": "What is your name?",
                    "hello": "Hello $1",
                    "added_to_database": "Hello $1, I've added you to the database!",
                    "database_contents": "Database contents: "
                }
            } );
            $('body').i18n();
            $('#user_name').attr("placeholder", $.i18n('name') );
        });
        </script>

</head>




<body>
    <div class="container" id="container">
       <h3 data-i18n="Accident Severity Prediction Model"></h3> 
        <h3></h3> 
        <h4 data-i18n="Police Force:"></h4> 
            <select name="police" method="GET" action="/" form="form">
            {% for police in pf_list %}
                <option value="{{police}}"{% if police==pf_sel %} selected="selected"{% endif %}>{{police}}</option>
            {% endfor %}
            </select>
        <h4 data-i18n={{cat_names[3]+":"}}></h4> 
            <select name="cat3" method="GET" action="/" form="form">
            {% for cat3 in cat_values[3] %}
                <option value="{{cat3}}"{% if cat3==cat_sel[3] %} selected="selected"{% endif %}>{{cat3}}</option>
            {% endfor %}
            </select>
        <h4 data-i18n={{cat_names[2]+":"}}></h4> 
            <select name="cat2" method="GET" action="/" form="form">
            {% for cat2 in cat_values[2] %}
                <option value="{{cat2}}"{% if cat2==cat_sel[2] %} selected="selected"{% endif %}>{{cat2}}</option>
            {% endfor %}
            </select>
        <h4 data-i18n={{cat_names[0]+":"}}></h4> 
            <select name="cat0" method="GET" action="/" form="form">
            {% for cat0 in cat_values[0] %}
                <option value="{{cat0}}"{% if cat0==cat_sel[0] %} selected="selected"{% endif %}>{{cat0}}</option>
            {% endfor %}
            </select>
        <h4 data-i18n={{cat_names[1]+":"}}></h4> 
            <select name="cat1" method="GET" action="/" form="form">
            {% for cat1 in cat_values[1] %}
                <option value="{{cat1}}"{% if cat1==cat_sel[1] %} selected="selected"{% endif %}>{{cat1}}</option>
            {% endfor %}
            </select>
        <h4 data-i18n={{cat_names[4]+":"}}></h4> 
            <select name="cat4" method="GET" action="/" form="form">
            {% for cat4 in cat_values[4] %}
                <option value="{{cat4}}"{% if cat4==cat_sel[4] %} selected="selected"{% endif %}>{{cat4}}</option>
            {% endfor %}
            </select>
        <h3> </h3> 
        <form id="form" action="{{ url_for('run_model') }}" method="POST">
            <input type="submit" value="Run Model!" id="model_button">
        </form>
        <h3> </h3> 
        <h5>Accident Severity Ratio Predicted </h5>
        <h5>by Gradient Boosted Trees Regressor </h5>
        <h5>Test Set Metrics: RMSE={{rmse}} - R2={{R2}}</h5>
     <img src="{{url_for('static', filename=img)}}" id="imgslot" style="position:absolute; TOP:45px; LEFT:450px; WIDTH:900px; HEIGHT:600px">
   </div>
    <footer class="footer">
        <div class="container">
            <span>IBM Coursera Advanced Data Science Capstone Project - Yann Adjanor Nov 2018</span>
        </div>
    </footer>

</body>


</html>

    <!--img src="data:image/png;base64,{{ img }}" id="imgslot" -->


        <!--img src="{{ img_url }}" id="imgslot" -->





## The file named "vcap-local.json" needs to be present in the current application directory
Fill it with the correct credentials

In [ ]:
{
  "services": {
    "cloudantNoSQLDB": [
      {
        "credentials": {
          "username":"XXXXXXXXXXXXXXXXXXXXXXX-bluemix",
          "password":"XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
          "host":"XXXXXXXXXXXXXXXXXXXXXXXX-bluemix.cloudant.com"
        },
        "label": "cloudantNoSQLDB"
      }
    ]
  }
}

<b>To provide initial setup, run and modify accordingly the following:</b><br>
<br>
git clone https://github.com/IBM-Cloud/get-started-python
<br>
cd get-started-python
